# Анализ выбора портфеля по коэффициенту Шарпа жадным алгоритмом

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime

Считываем данные

In [8]:
start = datetime.now()
data = pd.read_csv("Россия 37 Акций 2016-01.07.2019.txt")
data = np.array(data)
periods_in_data = data.shape[0]
print(periods_in_data)

881


In [9]:
print(data)

[[ 2.01601040e+07 -1.52180400e+00 -1.52155153e+00 ... -2.70986745e+00
   4.71380471e-01  1.11663653e+01]
 [ 2.01601050e+07  3.02138343e+00  1.12824001e+00 ...  3.70874962e+00
   2.37935657e+00  1.12647418e+01]
 [ 2.01601060e+07 -8.40336134e-02 -3.16966245e-01 ... -1.16771274e+00
  -5.72831424e-01  3.80116959e+00]
 ...
 [ 2.01906270e+07  4.01231688e-01  1.62874666e-01 ...  6.00200067e-01
   1.16510655e+00  1.03524229e+00]
 [ 2.01906280e+07 -1.39405204e+00 -4.96823352e-01 ... -4.97182632e-01
  -3.34678610e-01 -1.26444299e+00]
 [ 2.01907010e+07  1.38548539e+00  1.28785003e+00 ...  4.16389074e-01
  -6.81481481e-01  3.86398764e+00]]


In [12]:
data = np.delete(data, 0, axis = 1)
#data = np.delete(data, 0, axis = 1)  # Удалять дополительно второй столбец только в часовых котировках

stocks_to_portf_min = 5
stocks_to_portf_max = 6
stocks_to_portf_step = 1

sharp_period_min = 20
sharp_period_max = 320
sharp_period_step = 20

hold_period_min = 10       # Важна кратность диапазона end-start по hold(step)
hold_period_max = 190
hold_period_step = 10

sharp_period = 125
start_of_test_period = sharp_period_max
end_of_test_period = periods_in_data // hold_period_max * hold_period_max

mean_annual_profit_overal = []
P_L_overal = []
random_income_overal = []
random_portf_list = []
# Рассматриваем разное количество акций в портфеле по Кэф Шарпа
for stocks_to_portf in range(stocks_to_portf_min, stocks_to_portf_max, stocks_to_portf_step):
    # Рассматриваем разный период рассчета Кэф Шарпа.
    for sharp_period in range(sharp_period_min,sharp_period_max,sharp_period_step):
        print('sharp_period:', sharp_period)
        #Смотрим также результаты стратегии Купи и держи в те же периоды.
        P_L_by_hold = []
        mean_annual_profit_by_hold = []
        random_income_hold = []
        for hold in range(hold_period_min,hold_period_max,hold_period_step):
            print('hold:', hold)
            start_of_test_period = sharp_period_max # Должен быть больше чем sharp_period
            columns_count = data.shape[1] # число акций
            stocks_to_portf_list = []
            overal_results = []
            random_portf_list = []
            
            for i in range(start_of_test_period,end_of_test_period,hold):
            #for i in range(sharp_period,sharp_period+hold*10,hold):        # Пересчитываем через период hold коэфф Шарпа
                stocks_to_portf_list = []          
                counter = 1
                random_portf = np.mean(np.array(data[i-sharp_period:i,:]))
                random_array = np.array(data[i-sharp_period:i,:])
                random_portf_list.append(random_portf)
                for k in range(0, stocks_to_portf):
                    sharp_ratio_list = []
                    if counter == 1:                             # Выбираем первую акцию в портфель
                        for j in range(0, columns_count):
                            share_prices = np.array(data[i-sharp_period:i,j], dtype=float)
                            income = np.sum(share_prices)
                            std = np.std(share_prices)
                            if std > 0 and income > 0:
                                sharp_ratio = income/std
                            else:
                                sharp_ratio = 0
                            sharp_ratio_list.append(sharp_ratio)
                        
                        max_sharp_ratio = max(sharp_ratio_list)
                        indx_max_sharp_ratio = np.argmax(sharp_ratio_list)
                        comulative_income = np.array(data[i-sharp_period:i,indx_max_sharp_ratio], dtype=float)   # Инициализируем доход портфеля
                        stocks_to_portf_list.append(indx_max_sharp_ratio)
                        counter = counter + 1
                    else:
                        for j in range(0, columns_count):
                            if j in stocks_to_portf_list:    # Чтобы не включать акцию повторно
                                sharp_ratio_list.append(0)
                            else:
                                share_prices = np.array(data[i-sharp_period:i,j], dtype=float)
                                sub_sumpl_comulative_income = comulative_income + share_prices 
                                income = np.sum(sub_sumpl_comulative_income)
                                std = np.std(sub_sumpl_comulative_income)
                                if std > 0 and income > 0:
                                    sharp_ratio = income/std
                                else:
                                    sharp_ratio = 0
                                sharp_ratio_list.append(sharp_ratio)
                        
                        max_sharp_ratio = max(sharp_ratio_list)
                        indx_max_sharp_ratio = np.argmax(sharp_ratio_list)
                        comulative_income = comulative_income + np.array(data[i-sharp_period:i,indx_max_sharp_ratio], dtype=float)
                        stocks_to_portf_list.append(indx_max_sharp_ratio)
                        counter = counter + 1
                
                stocks_to_portf_list = np.array(stocks_to_portf_list)
                for j in range(0,stocks_to_portf):
                    if j == 0:
                        stock_ind = stocks_to_portf_list[j]
                        results = np.array(data[i:i+hold,stock_ind])#, dtype=float)
                    else:
                        stock_ind = stocks_to_portf_list[j]
                        results = results + np.array(data[i:i+hold,stock_ind])#, dtype=float)
                results = np.array(results)        
                overal_results.append(results)
            
            #overal_results = pd.DataFrame(overal_results)
            #print(overal_results)
            overal_results = np.array(overal_results)    
            
            
            # Анализируем результаты
            print(overal_results.shape)
            strings = overal_results.shape[0]
            rows  = overal_results.shape[1]
            
            mean_profit_list = []
            P_L_list = []
            
            for i in range(0, overal_results.shape[0]):
                profit = 0
                loss = 0
                for j in range(0, overal_results.shape[1]):
                    if overal_results[i][j] >= 0:
                        profit = profit + overal_results[i][j]
                    else:
                        loss = loss + overal_results[i][j]
                        
                mean_profit = (profit + loss)/stocks_to_portf
                P_L = -profit / loss
                mean_profit_list.append(mean_profit)
                P_L_list.append(P_L)
            
            mean_profit_list = np.array(mean_profit_list)
            P_L_list = np.array(P_L_list)
            random_portf_list = np.array(random_portf_list)
            
             
            #print(max_sharp_ratio, indx_max_sharp_ratio)
            #print(stocks_to_portf_list)
            #print(mean_profit_list)
            mean_annual_profit = mean_profit_list.mean()*(250 / hold)     # Менять когда час 2250 или День 250 
            P_L = P_L_list.mean()
            random_income = random_portf_list.mean() * 250              # Менять когда час 2250 или День 250
            
            mean_annual_profit_by_hold.append(mean_annual_profit)
            P_L_by_hold.append(P_L)
            random_income_hold.append(random_income)
            
            print('mean_annual_profit:', mean_annual_profit)
            #print(P_L_list)
            print('P_L_list.mean:', P_L)
            print('random_portf_mean_profit:', random_income)
            
        
        mean_annual_profit_by_hold = np.array(mean_annual_profit_by_hold)
        P_L_by_hold = np.array(P_L_by_hold)
        random_income_hold = np.array(random_income_hold)
        
        mean_annual_profit_overal.append(mean_annual_profit_by_hold)
        P_L_overal.append(P_L_by_hold)
        random_income_overal.append(random_income_hold)
        print('check')
            
mean_annual_profit_overal = np.array(mean_annual_profit_overal)       
P_L_overal = np.array(P_L_overal)
random_income_overal = np.array(random_income_overal)
finish = datetime.now()
print('start: ',start)
print('finish: ',finish)
print('difference: ', finish-start)    

print('mean_annual_profit_overal:', mean_annual_profit_overal)
print('P_L_overal:', P_L_overal)

mean_annual_profit_overal = pd.DataFrame(mean_annual_profit_overal)
P_L_overal = pd.DataFrame(P_L_overal)
random_income_overal = pd.DataFrame(random_income_overal)

print('mean_annual_profit_overal:', mean_annual_profit_overal)
print('P_L_overal:', P_L_overal)
print('random_income_overal:', random_income_overal)

finish = datetime.now()
time_consumed = finish - start
print('start:', start)
print('finish:', finish)
print('time_consumed:', time_consumed)


#random_max_list = []
#random_min_list = []
#for i in range(0,random_array.shape[0]):
#    random_max_list.append(np.max(random_array[i,:]))
#    random_min_list.append(np.min(random_array[i,:]))
#
#print('random_max_list:', random_max_list)
#print('random_min_list:', random_min_list)

sharp_period: 20
hold: 10
(44, 10)
mean_annual_profit: -5.243741351814699
P_L_list.mean: 1.2401538794802744
random_portf_mean_profit: 4.914264422445845
hold: 20
(22, 20)
mean_annual_profit: 8.576612534746502
P_L_list.mean: 1.2958040280361913
random_portf_mean_profit: 6.185747300458909
hold: 30
(15, 30)
mean_annual_profit: 17.819304281210677
P_L_list.mean: 1.4556307940633282
random_portf_mean_profit: 0.5886800595868036
hold: 40
(11, 40)
mean_annual_profit: 9.383847287421055
P_L_list.mean: 1.2915531731954955
random_portf_mean_profit: 4.25924599556142
hold: 50
(9, 50)
mean_annual_profit: 8.751969632699396
P_L_list.mean: 1.157563117581298
random_portf_mean_profit: -6.961022403221471
hold: 60
(8, 60)
mean_annual_profit: 13.316731448415522
P_L_list.mean: 1.2439962401698967
random_portf_mean_profit: -6.846240287846309
hold: 70
(7, 70)
mean_annual_profit: 17.033193039405166
P_L_list.mean: 1.3017081665563928
random_portf_mean_profit: 6.062783964938638
hold: 80
(6, 80)
mean_annual_profit: 12.979

(7, 70)
mean_annual_profit: 16.82341290701039
P_L_list.mean: 1.253938169424677
random_portf_mean_profit: 0.09476582315963
hold: 80
(6, 80)
mean_annual_profit: 15.512338131873024
P_L_list.mean: 1.2336734266429494
random_portf_mean_profit: 1.9146331098195264
hold: 90
(5, 90)
mean_annual_profit: 14.527883457277374
P_L_list.mean: 1.2025122733007416
random_portf_mean_profit: -4.535685460163906
hold: 100
(5, 100)
mean_annual_profit: 10.098531050436236
P_L_list.mean: 1.1829607972598197
random_portf_mean_profit: 0.0060743768754626185
hold: 110
(4, 110)
mean_annual_profit: 18.919410625348032
P_L_list.mean: 1.2970877497582536
random_portf_mean_profit: 9.560911846161817
hold: 120
(4, 120)
mean_annual_profit: 17.963594037406754
P_L_list.mean: 1.2544527154720666
random_portf_mean_profit: 0.38214680713857374
hold: 130
(4, 130)
mean_annual_profit: 19.890743000815714
P_L_list.mean: 1.2711606452324478
random_portf_mean_profit: 5.375098255066735
hold: 140
(4, 140)
mean_annual_profit: 19.51764540746629
P

(44, 10)
mean_annual_profit: 19.9882666179046
P_L_list.mean: 1.9910114622114703
random_portf_mean_profit: 6.403004463201327
hold: 20
(22, 20)
mean_annual_profit: 21.638659003556057
P_L_list.mean: 1.5188565887409773
random_portf_mean_profit: 6.837775313073274
hold: 30
(15, 30)
mean_annual_profit: 23.233687690118664
P_L_list.mean: 1.467968683582751
random_portf_mean_profit: 5.586873823330365
hold: 40
(11, 40)
mean_annual_profit: 21.525683182505897
P_L_list.mean: 1.3806023762131738
random_portf_mean_profit: 6.780710893488612
hold: 50
(9, 50)
mean_annual_profit: 14.3781922059183
P_L_list.mean: 1.3328956152442348
random_portf_mean_profit: 7.41227862053758
hold: 60
(8, 60)
mean_annual_profit: 20.857372971120668
P_L_list.mean: 1.3204460593542362
random_portf_mean_profit: 4.25681867943588
hold: 70
(7, 70)
mean_annual_profit: 19.224342634931396
P_L_list.mean: 1.2932151237859206
random_portf_mean_profit: 7.894572197653738
hold: 80
(6, 80)
mean_annual_profit: 15.640638347270164
P_L_list.mean: 1.2

(5, 100)
mean_annual_profit: 2.9614262621848297
P_L_list.mean: 1.0706486225620522
random_portf_mean_profit: 13.812887135163807
hold: 110
(4, 110)
mean_annual_profit: 8.125863187883498
P_L_list.mean: 1.12724603185978
random_portf_mean_profit: 14.409668128742835
hold: 120
(4, 120)
mean_annual_profit: 8.098224991019194
P_L_list.mean: 1.114468338812811
random_portf_mean_profit: 11.138948292188136
hold: 130
(4, 130)
mean_annual_profit: 10.787636309077314
P_L_list.mean: 1.1287430619182492
random_portf_mean_profit: 11.781202202243277
hold: 140
(4, 140)
mean_annual_profit: 20.704136841688076
P_L_list.mean: 1.2886463308295943
random_portf_mean_profit: 9.283269155083946
hold: 150
(3, 150)
mean_annual_profit: 9.204315339981921
P_L_list.mean: 1.1307281514663565
random_portf_mean_profit: 12.916212760347907
hold: 160
(3, 160)
mean_annual_profit: 8.874315152121088
P_L_list.mean: 1.1156224566336863
random_portf_mean_profit: 9.542426588151248
hold: 170
(3, 170)
mean_annual_profit: 6.176078529672809
P_L

(44, 10)
mean_annual_profit: 12.213290291906507
P_L_list.mean: 1.5615971146989742
random_portf_mean_profit: 12.049088002863055
hold: 20
(22, 20)
mean_annual_profit: 13.28231097607401
P_L_list.mean: 1.3965476086356776
random_portf_mean_profit: 12.445669036487054
hold: 30
(15, 30)
mean_annual_profit: 11.783853164101746
P_L_list.mean: 1.2196964929703387
random_portf_mean_profit: 12.00005439961531
hold: 40
(11, 40)
mean_annual_profit: 10.270235167167636
P_L_list.mean: 1.1841331231371224
random_portf_mean_profit: 13.005275104257407
hold: 50
(9, 50)
mean_annual_profit: 8.822678923606201
P_L_list.mean: 1.1458023972886684
random_portf_mean_profit: 12.759996849525425
hold: 60
(8, 60)
mean_annual_profit: 8.60352266088115
P_L_list.mean: 1.1022903472161087
random_portf_mean_profit: 12.127593563814415
hold: 70
(7, 70)
mean_annual_profit: 6.168478688667679
P_L_list.mean: 1.0729738621164813
random_portf_mean_profit: 13.388666923156
hold: 80
(6, 80)
mean_annual_profit: 10.441177764780036
P_L_list.mean